In [31]:
# Dependencies 
import numpy as np
import pandas as pd
from collections import Counter
from pathlib import Path
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import accuracy_score
from sklearn.utils import resample
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix, classification_report, roc_curve, auc

In [32]:
# Bring in 2019loans as train_df
train_df = pd.read_csv(Path('Resources/2019loans.csv'))

# Bring in 2020Q1loans as test_df
test_df = pd.read_csv(Path('Resources/2020Q1loans.csv'))

# Display for check
display(train_df)
display(test_df)

,loan_amnt,int_rate,installment,home_ownership,annual_inc,verification_status,pymnt_plan,dti,delinq_2yrs,inq_last_6mths,...,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,hardship_flag,debt_settlement_flag,target
0,7000.0,0.1894,256.38,MORTGAGE,75000.0,Not Verified,n,28.62,0.0,2.0,...,87.5,0.0,0.0,352260.0,62666.0,35000.0,10000.0,N,N,low_risk
1,40000.0,0.1614,975.71,MORTGAGE,102000.0,Source Verified,n,11.72,2.0,0.0,...,0.0,0.0,0.0,294664.0,109911.0,9000.0,71044.0,N,N,low_risk
2,11000.0,0.2055,294.81,RENT,45000.0,Verified,n,37.25,1.0,3.0,...,7.7,0.0,0.0,92228.0,36007.0,33000.0,46328.0,N,N,low_risk
3,4000.0,0.1612,140.87,MORTGAGE,38000.0,Not Verified,n,42.89,1.0,0.0,...,100.0,0.0,0.0,284273.0,52236.0,13500.0,52017.0,N,N,low_risk
4,14000.0,0.1797,505.93,MORTGAGE,43000.0,Source Verified,n,22.16,1.0,0.0,...,25.0,0.0,0.0,120280.0,88147.0,33300.0,78680.0,N,N,low_risk
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12175,19975.0,0.2565,801.09,RENT,28000.0,Not Verified,n,28.42,0.0,0.0,...,16.7,0.0,0.0,50055.0,28192.0,18700.0,19055.0,N,N,high_risk
12176,15000.0,0.1774,540.34,RENT,50000.0,Verified,n,23.43,4.0,0.0,...,11.1,0.0,0.0,70324.0,57025.0,13300.0,54824.0,N,N,high_risk
12177,3600.0,0.1862,131.28,RENT,60000.0,Not Verified,n,28.80,0.0,1.0,...,0.0,0.0,0.0,83765.0,55156.0,14800.0,53065.0,N,N,high_risk
12178,15000.0,0.0881,475.68,MORTGAGE,62000.0,Source Verified,n,11.44,0.0,0.0,...,0.0,0.0,0.0,189930.0,23748.0,7000.0,32930.0,N,N,high_risk


,loan_amnt,int_rate,installment,home_ownership,annual_inc,verification_status,pymnt_plan,dti,delinq_2yrs,inq_last_6mths,...,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,hardship_flag,debt_settlement_flag,target
0,40000.0,0.1033,856.40,RENT,128700.0,Source Verified,n,12.47,0.0,1.0,...,57.1,0.0,0.0,63915.0,49510.0,49400.0,14515.0,Y,N,low_risk
1,24450.0,0.1430,572.72,MORTGAGE,44574.0,Not Verified,n,15.05,0.0,1.0,...,0.0,0.0,0.0,136425.0,19439.0,15500.0,18925.0,N,N,low_risk
2,13500.0,0.1430,316.23,OWN,60000.0,Not Verified,n,28.72,0.0,0.0,...,0.0,0.0,0.0,82124.0,65000.0,5400.0,61724.0,Y,N,low_risk
3,10625.0,0.1774,268.31,RENT,60000.0,Verified,n,15.70,0.0,4.0,...,20.0,0.0,0.0,54855.0,50335.0,23200.0,26255.0,N,N,low_risk
4,6375.0,0.1862,232.46,RENT,60000.0,Source Verified,n,35.50,0.0,0.0,...,75.0,0.0,0.0,90445.0,56541.0,15300.0,72345.0,N,N,low_risk
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4697,30000.0,0.1240,673.42,RENT,140480.0,Source Verified,n,15.74,0.0,0.0,...,28.6,0.0,0.0,159688.0,110873.0,48400.0,107388.0,N,N,high_risk
4698,24000.0,0.0756,747.22,RENT,50000.0,Not Verified,n,26.81,0.0,0.0,...,0.0,0.0,0.0,62375.0,18928.0,13300.0,30775.0,N,N,high_risk
4699,10000.0,0.2305,387.36,RENT,33000.0,Verified,n,38.51,0.0,2.0,...,0.0,0.0,0.0,43250.0,33022.0,8500.0,29550.0,N,N,high_risk
4700,8000.0,0.1862,205.86,RENT,38000.0,Source Verified,n,16.36,0.0,1.0,...,0.0,1.0,0.0,31357.0,19595.0,1500.0,9657.0,N,N,high_risk


## Preprocessing 
#### Convert categorical data to numeric and separate target feature for training data

In [33]:
### Create training set from 2019 loans (train_df)

# Drop 'target' column to create X data,
# Use the 'target' column for the y data
X_train = train_df.drop('target', axis=1)
y_train = train_df['target']

# Convert categorical X data to numeric using pd.get_dummies()
# Convert categorical y data to numeric using LabelEncoder().fit_transform() -- low-risk = 1, high-risk = 0
X_train_numeric = pd.get_dummies(X_train)
y_train_numeric = LabelEncoder().fit_transform(y_train)

# Display for check
print(X_train_numeric.columns)
display(X_train_numeric)
display(y_train_numeric)

Index(['loan_amnt', 'int_rate', 'installment', 'annual_inc', 'dti',
       'delinq_2yrs', 'inq_last_6mths', 'open_acc', 'pub_rec', 'revol_bal',
       'total_acc', 'out_prncp', 'out_prncp_inv', 'total_pymnt',
       'total_pymnt_inv', 'total_rec_prncp', 'total_rec_int',
       'total_rec_late_fee', 'recoveries', 'collection_recovery_fee',
       'last_pymnt_amnt', 'collections_12_mths_ex_med', 'policy_code',
       'acc_now_delinq', 'tot_coll_amt', 'tot_cur_bal', 'open_acc_6m',
       'open_act_il', 'open_il_12m', 'open_il_24m', 'mths_since_rcnt_il',
       'total_bal_il', 'il_util', 'open_rv_12m', 'open_rv_24m', 'max_bal_bc',
       'all_util', 'total_rev_hi_lim', 'inq_fi', 'total_cu_tl', 'inq_last_12m',
       'acc_open_past_24mths', 'avg_cur_bal', 'bc_open_to_buy', 'bc_util',
       'chargeoff_within_12_mths', 'delinq_amnt', 'mo_sin_old_il_acct',
       'mo_sin_old_rev_tl_op', 'mo_sin_rcnt_rev_tl_op', 'mo_sin_rcnt_tl',
       'mort_acc', 'mths_since_recent_bc', 'mths_since_recent_in

,loan_amnt,int_rate,installment,annual_inc,dti,delinq_2yrs,inq_last_6mths,open_acc,pub_rec,revol_bal,...,verification_status_Verified,pymnt_plan_n,initial_list_status_f,initial_list_status_w,application_type_Individual,application_type_Joint App,hardship_flag_N,hardship_flag_Y,debt_settlement_flag_N,debt_settlement_flag_Y
0,7000.0,0.1894,256.38,75000.0,28.62,0.0,2.0,20.0,0.0,40414.0,...,0,1,1,0,1,0,1,0,1,0
1,40000.0,0.1614,975.71,102000.0,11.72,2.0,0.0,10.0,0.0,43531.0,...,0,1,0,1,1,0,1,0,1,0
2,11000.0,0.2055,294.81,45000.0,37.25,1.0,3.0,23.0,0.0,8242.0,...,1,1,0,1,1,0,1,0,1,0
3,4000.0,0.1612,140.87,38000.0,42.89,1.0,0.0,7.0,0.0,12767.0,...,0,1,0,1,0,1,1,0,1,0
4,14000.0,0.1797,505.93,43000.0,22.16,1.0,0.0,22.0,0.0,11182.0,...,0,1,0,1,1,0,1,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12175,19975.0,0.2565,801.09,28000.0,28.42,0.0,0.0,15.0,0.0,9679.0,...,0,1,0,1,1,0,1,0,1,0
12176,15000.0,0.1774,540.34,50000.0,23.43,4.0,0.0,16.0,0.0,3193.0,...,1,1,0,1,1,0,1,0,1,0
12177,3600.0,0.1862,131.28,60000.0,28.80,0.0,1.0,14.0,0.0,11804.0,...,0,1,0,1,1,0,1,0,1,0
12178,15000.0,0.0881,475.68,62000.0,11.44,0.0,0.0,5.0,0.0,157.0,...,0,1,0,1,0,1,1,0,1,0


array([1, 1, 1, ..., 0, 0, 0])

In [34]:
### Create testing set from 2020 loans (test_df)

# Drop 'target' column to create X data,
# Use the 'target' column for the y data
X_test = test_df.drop('target', axis=1)
y_test = test_df['target']

# Convert categorical X data to numeric using pd.get_dummies()
# Convert categorical y data to numeric using LabelEncoder().fit_transform() -- low-risk = 1, high-risk = 0
X_test_numeric = pd.get_dummies(X_test)
y_test_numeric = LabelEncoder().fit_transform(y_test)

# Display for check
print(X_test_numeric.columns)
display(X_test_numeric)
display(y_test_numeric)

Index(['loan_amnt', 'int_rate', 'installment', 'annual_inc', 'dti',
       'delinq_2yrs', 'inq_last_6mths', 'open_acc', 'pub_rec', 'revol_bal',
       'total_acc', 'out_prncp', 'out_prncp_inv', 'total_pymnt',
       'total_pymnt_inv', 'total_rec_prncp', 'total_rec_int',
       'total_rec_late_fee', 'recoveries', 'collection_recovery_fee',
       'last_pymnt_amnt', 'collections_12_mths_ex_med', 'policy_code',
       'acc_now_delinq', 'tot_coll_amt', 'tot_cur_bal', 'open_acc_6m',
       'open_act_il', 'open_il_12m', 'open_il_24m', 'mths_since_rcnt_il',
       'total_bal_il', 'il_util', 'open_rv_12m', 'open_rv_24m', 'max_bal_bc',
       'all_util', 'total_rev_hi_lim', 'inq_fi', 'total_cu_tl', 'inq_last_12m',
       'acc_open_past_24mths', 'avg_cur_bal', 'bc_open_to_buy', 'bc_util',
       'chargeoff_within_12_mths', 'delinq_amnt', 'mo_sin_old_il_acct',
       'mo_sin_old_rev_tl_op', 'mo_sin_rcnt_rev_tl_op', 'mo_sin_rcnt_tl',
       'mort_acc', 'mths_since_recent_bc', 'mths_since_recent_in

,loan_amnt,int_rate,installment,annual_inc,dti,delinq_2yrs,inq_last_6mths,open_acc,pub_rec,revol_bal,...,verification_status_Source Verified,verification_status_Verified,pymnt_plan_n,initial_list_status_f,initial_list_status_w,application_type_Individual,application_type_Joint App,hardship_flag_N,hardship_flag_Y,debt_settlement_flag_N
0,40000.0,0.1033,856.40,128700.0,12.47,0.0,1.0,8.0,0.0,38113.0,...,1,0,1,0,1,1,0,0,1,1
1,24450.0,0.1430,572.72,44574.0,15.05,0.0,1.0,6.0,0.0,1665.0,...,0,0,1,0,1,1,0,1,0,1
2,13500.0,0.1430,316.23,60000.0,28.72,0.0,0.0,8.0,0.0,13857.0,...,0,0,1,0,1,1,0,0,1,1
3,10625.0,0.1774,268.31,60000.0,15.70,0.0,4.0,17.0,0.0,6216.0,...,0,1,1,0,1,1,0,1,0,1
4,6375.0,0.1862,232.46,60000.0,35.50,0.0,0.0,13.0,0.0,12681.0,...,1,0,1,0,1,1,0,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4697,30000.0,0.1240,673.42,140480.0,15.74,0.0,0.0,20.0,0.0,23215.0,...,1,0,1,1,0,1,0,1,0,1
4698,24000.0,0.0756,747.22,50000.0,26.81,0.0,0.0,9.0,0.0,459.0,...,0,0,1,0,1,1,0,1,0,1
4699,10000.0,0.2305,387.36,33000.0,38.51,0.0,2.0,7.0,0.0,6342.0,...,0,1,1,1,0,1,0,1,0,1
4700,8000.0,0.1862,205.86,38000.0,16.36,0.0,1.0,8.0,1.0,11636.0,...,1,0,1,0,1,1,0,1,0,1


array([1, 1, 1, ..., 0, 0, 0])

In [35]:
# Check that columns for training and test data are the same
print(f"columns shared: {X_train_numeric.columns.intersection(X_test_numeric.columns)}")
print(f"columns not shared: {set(X_train_numeric.columns).symmetric_difference(X_test_numeric.columns)}")

columns shared: Index(['loan_amnt', 'int_rate', 'installment', 'annual_inc', 'dti',
       'delinq_2yrs', 'inq_last_6mths', 'open_acc', 'pub_rec', 'revol_bal',
       'total_acc', 'out_prncp', 'out_prncp_inv', 'total_pymnt',
       'total_pymnt_inv', 'total_rec_prncp', 'total_rec_int',
       'total_rec_late_fee', 'recoveries', 'collection_recovery_fee',
       'last_pymnt_amnt', 'collections_12_mths_ex_med', 'policy_code',
       'acc_now_delinq', 'tot_coll_amt', 'tot_cur_bal', 'open_acc_6m',
       'open_act_il', 'open_il_12m', 'open_il_24m', 'mths_since_rcnt_il',
       'total_bal_il', 'il_util', 'open_rv_12m', 'open_rv_24m', 'max_bal_bc',
       'all_util', 'total_rev_hi_lim', 'inq_fi', 'total_cu_tl', 'inq_last_12m',
       'acc_open_past_24mths', 'avg_cur_bal', 'bc_open_to_buy', 'bc_util',
       'chargeoff_within_12_mths', 'delinq_amnt', 'mo_sin_old_il_acct',
       'mo_sin_old_rev_tl_op', 'mo_sin_rcnt_rev_tl_op', 'mo_sin_rcnt_tl',
       'mort_acc', 'mths_since_recent_bc', 'mths

In [36]:
# Add missing column in testing data
X_test_numeric['debt_settlement_flag_Y'] = 0

# Check training and testing data columns again
print(f"columns shared: {X_train_numeric.columns.intersection(X_test_numeric.columns)}")
print(f"columns not shared: {set(X_train_numeric.columns).symmetric_difference(X_test_numeric.columns)}")

columns shared: Index(['loan_amnt', 'int_rate', 'installment', 'annual_inc', 'dti',
       'delinq_2yrs', 'inq_last_6mths', 'open_acc', 'pub_rec', 'revol_bal',
       'total_acc', 'out_prncp', 'out_prncp_inv', 'total_pymnt',
       'total_pymnt_inv', 'total_rec_prncp', 'total_rec_int',
       'total_rec_late_fee', 'recoveries', 'collection_recovery_fee',
       'last_pymnt_amnt', 'collections_12_mths_ex_med', 'policy_code',
       'acc_now_delinq', 'tot_coll_amt', 'tot_cur_bal', 'open_acc_6m',
       'open_act_il', 'open_il_12m', 'open_il_24m', 'mths_since_rcnt_il',
       'total_bal_il', 'il_util', 'open_rv_12m', 'open_rv_24m', 'max_bal_bc',
       'all_util', 'total_rev_hi_lim', 'inq_fi', 'total_cu_tl', 'inq_last_12m',
       'acc_open_past_24mths', 'avg_cur_bal', 'bc_open_to_buy', 'bc_util',
       'chargeoff_within_12_mths', 'delinq_amnt', 'mo_sin_old_il_acct',
       'mo_sin_old_rev_tl_op', 'mo_sin_rcnt_rev_tl_op', 'mo_sin_rcnt_tl',
       'mort_acc', 'mths_since_recent_bc', 'mths

## Consider the Models: Logistic Regression vs. Random Forest Classifer

#### Predictions for which model will perform better: 
I expect that the Random Forest Classifier will perform better than the Logistic Regression for this dataset. Since there are a large number of variables included, some may be more predictive while others may be noise; an RFC would be able to isolate the more predictive variables to hopefully improve the accuracy of the results. RFCs may also be better suited to handle the categorical variables included (several of which have more than 2 possible values) since this reduces the likelyhood of the model having a linear relationship 

## Model Generation with Unscaled Data
#### Train the Logistic Regression and Random Forest Classifier models with the unscaled data and print the model scores

In [37]:
### Logistic Regression model

# Create the log. regression model -- need to increase max iterations for solver algorithm to converge
classifier_unscaled = LogisticRegression(max_iter=9000)

# Fit/train model using training data
classifier_unscaled.fit(X_train_numeric, y_train_numeric)

# Validate the model using the test data
print(f"Training Data Score: {classifier_unscaled.score(X_train_numeric, y_train_numeric)}")
print(f"Testing Data Score: {classifier_unscaled.score(X_test_numeric, y_test_numeric)}")


Training Data Score: 0.7059934318555008
Testing Data Score: 0.5612505316886431


In [38]:
### Random Forest Classifier model 

# Create the RFC model, fit to training data
clf_unscaled = RandomForestClassifier(random_state=1, n_estimators=500).fit(X_train_numeric, y_train_numeric)

# Validate the model using the test data
print(f"Training Data Score: {clf_unscaled.score(X_train_numeric, y_train_numeric)}")
print(f"Testing Data Score: {clf_unscaled.score(X_test_numeric, y_test_numeric)}")


Training Data Score: 1.0
Testing Data Score: 0.646958740961293


In [39]:
### Comparing Model Performance

# Make 'true' y data from y_test_numeric to use for both models' performance analysis
y_true = y_test_numeric

# ---- Logistic Regression model ----

# Get prediction values for the confusion matrix and classification reports
y_pred_LOG = classifier_unscaled.predict(X_test_numeric)
tn, fp, fn, tp = confusion_matrix(y_true, y_pred_LOG).ravel()

# Run receiver operating characteristics curve to find area under the curve 
y_proba_LOG = classifier_unscaled.predict_proba(X_test_numeric)[:,1]
fpr, tpr, thresholds = roc_curve(y_test_numeric, y_proba_LOG)
auc_LOG = auc(fpr, tpr)

# Diplay results of confusion matrix, classification report, and AUC 
print(f"Confusion Matrix for Log. Reg. model:")
print(f"{confusion_matrix(y_true, y_pred_LOG)}")
print(f"true positives: {tp}")
print(f"true negatives: {tn}")
print(f"false negatives: {fn}")
print(f"false positives: {fp}")
print(f"Classification Report for Log. Reg. model:")
print(classification_report(y_true, y_pred_LOG))
print(f"Area Under the Curve for Log. Reg. model:")
print(auc_LOG)

print(" ")
print("------------------------------------------------------")
print(" ")

# ---- Random Forest Classifier model ----

# Get prediction values for the confusion matrix and classification reports
y_pred_RFC = clf_unscaled.predict(X_test_numeric)
tn, fp, fn, tp = confusion_matrix(y_true, y_pred_RFC).ravel()

# Run receiver operating characteristics curve to find area under the curve 
y_proba_RFC = clf_unscaled.predict_proba(X_test_numeric)[:,1]
fpr, tpr, thresholds = roc_curve(y_test_numeric, y_proba_RFC)
auc_RFC = auc(fpr, tpr)

# Diplay results
print(f"Confusion Matrix for RFC model:")
print(f"{confusion_matrix(y_true, y_pred_RFC)}")
print(f"true positives: {tp}")
print(f"true negatives: {tn}")
print(f"false negatives: {fn}")
print(f"false positives: {fp}")
print(f"Classification Report for RFC model:")
print(classification_report(y_true, y_pred_RFC))
print(f"Area Under the Curve for RFC model:")
print(auc_RFC)

Confusion Matrix for Log. Reg. model:
[[ 821 1530]
 [ 533 1818]]
true positives: 1818
true negatives: 821
false negatives: 533
false positives: 1530
Classification Report for Log. Reg. model:
              precision    recall  f1-score   support

           0       0.61      0.35      0.44      2351
           1       0.54      0.77      0.64      2351

    accuracy                           0.56      4702
   macro avg       0.57      0.56      0.54      4702
weighted avg       0.57      0.56      0.54      4702

Area Under the Curve for Log. Reg. model:
0.43606610289728925
 
------------------------------------------------------
 
Confusion Matrix for RFC model:
[[1961  390]
 [1270 1081]]
true positives: 1081
true negatives: 1961
false negatives: 1270
false positives: 390
Classification Report for RFC model:
              precision    recall  f1-score   support

           0       0.61      0.83      0.70      2351
           1       0.73      0.46      0.57      2351

    accuracy   

#### Analyze model performance and compare to prediction

As predicted, the Random Forest Classifier performed better than the Logistic Regression model for this dataset (at least pre-scaling). While the logistic regression model returned a testing score/accuracy of 0.56 (training score of .71), the RFC model had an accuracy score of .65 (training score of 1.0). Additionally, the area under the curve for the RFC model was .70, indicating the model performed acceptably well; the logistic regression model, however, had an AUC of .44, meaning it performed worse than if we had flipped a coin.

Interestingly, it seems that the logistic regression was generally better at classifying low-risk cases (1s, f1-score of .64 vs .44) while the RFC was better at classifying the high-risk cases (0s, f1-score of .70 vs .57). While the LR model's precision was slightly higher for high-risk cases (.61) than for low-risk cases (.54), it's recall/sensitivity score was much higher for low-risk (.77) than for high-risk cases (.35) - in other words, it seemed better at classifying low-risk cases correctly as low-risk, while it missed and incorrectly classified about two-thirds of the high-risk cases. The opposite was true for the RFC model, which has a slightly higher precision score for low-risk (.73) than for high-risk cases (.61), but had a much higher recall score for high-risk (.83) than for low-risk cases (.46) - in other words, it correctly classified most of the truely high-risk cases in the dataset but misclassified over half of the low-risk cases. Considering the model would be used for investors to predict the credit risk level of future loans they may want to fund, the sensitivity scores for the models may be something they would want to consider, seeing as the better-performing RFC model may result in more conservative investments.

## Scaling the Data and Refitting the Models

#### Predictions for how scaling will affect model accuracy:
Based on my research (references below), since tree-based algorithms don't require scaling or normalization as they are not sensitive to the scales of the variables used, I don't think that scaling the data will affect the RFC model's results for this dataset much if at all. However, logistic regression algorithms require feature normalization so that numeric variables with wider ranges are treated equally by the model rather than treated as more important than those with shorther ranges. Since the majority of the variables in this dataset are numeric, with some having wider ranges (loan_amnt, revol_bal) than others (int_rate, dti), scaling the data will likely improve the results of the logistic regression model. Whether it will improve enough to match the RFC's results, however, is unclear. 

References: 

- https://towardsdatascience.com/all-about-feature-scaling-bcc0ad75cb35
- https://www.analyticsvidhya.com/blog/2020/04/feature-scaling-machine-learning-normalization-standardization/
- https://scikit-learn.org/stable/auto_examples/preprocessing/plot_scaling_importance.html

#### Train the Logistic Regression and Random Forest Classifier models with scaled data and print the model scores

In [40]:
### Find numeric data columns from original dataset
### **categorical variables/ one-hot encoded features should not be standardized!

# Put column data types into a list, use Counter() to count the number of columns for each data type
col_types = train_df.dtypes.tolist()
print(Counter(col_types))

# Separate numeric and categorical columns into lists using select_dtypes()
num_cols = train_df.select_dtypes(include=['float64'], exclude=['O']).columns.tolist()
cat_cols = train_df.select_dtypes(include=['O'], exclude=['float64']).columns.tolist()

# Check that each list has the correct variables included
print(num_cols)
print(len(num_cols))
print(cat_cols)
print(len(cat_cols))


Counter({dtype('float64'): 76, dtype('O'): 8})
['loan_amnt', 'int_rate', 'installment', 'annual_inc', 'dti', 'delinq_2yrs', 'inq_last_6mths', 'open_acc', 'pub_rec', 'revol_bal', 'total_acc', 'out_prncp', 'out_prncp_inv', 'total_pymnt', 'total_pymnt_inv', 'total_rec_prncp', 'total_rec_int', 'total_rec_late_fee', 'recoveries', 'collection_recovery_fee', 'last_pymnt_amnt', 'collections_12_mths_ex_med', 'policy_code', 'acc_now_delinq', 'tot_coll_amt', 'tot_cur_bal', 'open_acc_6m', 'open_act_il', 'open_il_12m', 'open_il_24m', 'mths_since_rcnt_il', 'total_bal_il', 'il_util', 'open_rv_12m', 'open_rv_24m', 'max_bal_bc', 'all_util', 'total_rev_hi_lim', 'inq_fi', 'total_cu_tl', 'inq_last_12m', 'acc_open_past_24mths', 'avg_cur_bal', 'bc_open_to_buy', 'bc_util', 'chargeoff_within_12_mths', 'delinq_amnt', 'mo_sin_old_il_acct', 'mo_sin_old_rev_tl_op', 'mo_sin_rcnt_rev_tl_op', 'mo_sin_rcnt_tl', 'mort_acc', 'mths_since_recent_bc', 'mths_since_recent_inq', 'num_accts_ever_120_pd', 'num_actv_bc_tl', 'nu

In [44]:
### Scale the X training and testing data using StandardScaler

# Make copies of the X training and testing datasets
X_train_scaled = X_train_numeric.copy()
X_test_scaled = X_test_numeric.copy()

# Apply StandardScaler on numeric feature columns
for i in num_cols:
    
    # create scaler to fit on training data column
    scaler = StandardScaler().fit(X_train_scaled[[i]])
    
    # transform the X training and testing data columns
    X_train_scaled[i] = scaler.transform(X_train_scaled[[i]])
    X_test_scaled[i] = scaler.transform(X_test_scaled[[i]])

# Display scaled datasets to check
display(X_train_scaled)
display(X_test_scaled)

,loan_amnt,int_rate,installment,annual_inc,dti,delinq_2yrs,inq_last_6mths,open_acc,pub_rec,revol_bal,...,verification_status_Verified,pymnt_plan_n,initial_list_status_f,initial_list_status_w,application_type_Individual,application_type_Joint App,hardship_flag_N,hardship_flag_Y,debt_settlement_flag_N,debt_settlement_flag_Y
0,-1.008699,0.919620,-0.850420,-0.104831,0.230684,-0.321620,1.758968,1.217675,-0.348464,0.997756,...,0,1,1,0,1,0,1,0,1,0
1,2.224099,0.387179,1.602612,0.109872,-0.437176,2.255569,-0.694141,-0.436716,-0.348464,1.133413,...,0,1,0,1,1,0,1,0,1,0
2,-0.616844,1.225773,-0.719368,-0.343389,0.571727,0.966975,2.985522,1.713993,-0.348464,-0.402418,...,1,1,0,1,1,0,1,0,1,0
3,-1.302589,0.383376,-1.244328,-0.399053,0.794611,0.966975,-0.694141,-0.933033,-0.348464,-0.205483,...,0,1,0,1,0,1,1,0,1,0
4,-0.322954,0.735167,0.000586,-0.359293,-0.024605,0.966975,-0.694141,1.548554,-0.348464,-0.274465,...,0,1,0,1,1,0,1,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12175,0.262378,2.195577,1.007129,-0.478573,0.222780,-0.321620,-0.694141,0.390480,-0.348464,-0.339878,...,0,1,0,1,1,0,1,0,1,0
12176,-0.224990,0.691431,0.117930,-0.303630,0.025584,4.832758,-0.694141,0.555919,-0.348464,-0.622158,...,1,1,0,1,1,0,1,0,1,0
12177,-1.341775,0.858769,-1.277032,-0.224110,0.237797,-0.321620,0.532413,0.225041,-0.348464,-0.247395,...,0,1,0,1,1,0,1,0,1,0
12178,-0.224990,-1.006676,-0.102572,-0.208206,-0.448241,-0.321620,-0.694141,-1.263911,-0.348464,-0.754290,...,0,1,0,1,0,1,1,0,1,0


,loan_amnt,int_rate,installment,annual_inc,dti,delinq_2yrs,inq_last_6mths,open_acc,pub_rec,revol_bal,...,verification_status_Verified,pymnt_plan_n,initial_list_status_f,initial_list_status_w,application_type_Individual,application_type_Joint App,hardship_flag_N,hardship_flag_Y,debt_settlement_flag_N,debt_settlement_flag_Y
0,2.224099,-0.717637,1.195746,0.322189,-0.407538,-0.32162,0.532413,-0.767594,-0.348464,0.897613,...,0,1,0,1,1,0,0,1,1,0
1,0.700765,0.037289,0.228351,-0.346777,-0.305580,-0.32162,0.532413,-1.098472,-0.348464,-0.688659,...,0,1,0,1,1,0,1,0,1,0
2,-0.371936,0.037289,-0.646322,-0.224110,0.234636,-0.32162,-0.694141,-0.767594,-0.348464,-0.158045,...,0,1,0,1,1,0,0,1,1,0
3,-0.653581,0.691431,-0.809737,-0.224110,-0.279893,-0.32162,4.212076,0.721358,-0.348464,-0.490593,...,1,1,0,1,1,0,1,0,1,0
4,-1.069926,0.858769,-0.931991,-0.224110,0.502570,-0.32162,-0.694141,0.059602,-0.348464,-0.209226,...,0,1,0,1,1,0,1,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4697,1.244463,-0.324010,0.571754,0.415863,-0.278313,-0.32162,-0.694141,1.217675,-0.348464,0.249230,...,0,1,1,0,1,0,1,0,1,0
4698,0.656682,-1.244373,0.823424,-0.303630,0.159156,-0.32162,-0.694141,-0.602155,-0.348464,-0.741146,...,0,1,0,1,1,0,1,0,1,0
4699,-0.714808,1.701167,-0.403757,-0.438813,0.621520,-0.32162,1.758968,-0.933033,-0.348464,-0.485109,...,1,1,1,0,1,0,1,0,1,0
4700,-0.910735,0.858769,-1.022702,-0.399053,-0.253811,-0.32162,0.532413,-0.767594,2.727106,-0.254706,...,0,1,0,1,1,0,1,0,1,0


In [45]:
### Logistic Regression model

# Create the log. regression model -- need to increase max iterations for solver algorithm to converge
classifier_scaled = LogisticRegression(max_iter=9000)

# Fit/train model using training data
classifier_scaled.fit(X_train_scaled, y_train_numeric)

# Validate the model using the test data
print(f"Training Data Score: {classifier_scaled.score(X_train_scaled, y_train_numeric)}")
print(f"Testing Data Score: {classifier_scaled.score(X_test_scaled, y_test_numeric)}")


Training Data Score: 0.7107553366174055
Testing Data Score: 0.7575499787324542


In [46]:
### Random Forest Classifier model 

# Create the RFC model, fit to training data
clf_scaled = RandomForestClassifier(random_state=1, n_estimators=500).fit(X_train_scaled, y_train_numeric)

# Validate the model using the test data
print(f"Training Data Score: {clf_scaled.score(X_train_scaled, y_train_numeric)}")
print(f"Testing Data Score: {clf_scaled.score(X_test_scaled, y_test_numeric)}")


Training Data Score: 1.0
Testing Data Score: 0.6480221182475542


In [47]:
### Comparing Model Performance

# Make 'true' y data from y_test_numeric to use for both models' performance analysis
y_true = y_test_numeric

# ---- Logistic Regression model ----

# Get prediction values for the confusion matrix and classification reports
y_pred_LOG_S = classifier_scaled.predict(X_test_scaled)
tn, fp, fn, tp = confusion_matrix(y_true, y_pred_LOG_S).ravel()

# Run receiver operating characteristics curve to find area under the curve 
y_proba_LOG_S = classifier_scaled.predict_proba(X_test_scaled)[:,1]
fpr, tpr, thresholds = roc_curve(y_test_numeric, y_proba_LOG_S)
auc_LOG_S = auc(fpr, tpr)

# Diplay results of confusion matrix, classification report, and AUC 
print(f"Confusion Matrix for Scaled Log. Reg. model:")
print(f"{confusion_matrix(y_true, y_pred_LOG_S)}")
print(f"true positives: {tp}")
print(f"true negatives: {tn}")
print(f"false negatives: {fn}")
print(f"false positives: {fp}")
print(f"Classification Report Scaled Log. Reg. model:")
print(classification_report(y_true, y_pred_LOG_S))
print(f"Area Under the Curve Scaled Log. Reg. model:")
print(auc_LOG_S)

print(" ")
print("------------------------------------------------------")
print(" ")

# ---- Random Forest Classifier model ----

# Get prediction values for the confusion matrix and classification reports
y_pred_RFC_S = clf_scaled.predict(X_test_scaled)
tn, fp, fn, tp = confusion_matrix(y_true, y_pred_RFC_S).ravel()

# Run receiver operating characteristics curve to find area under the curve 
y_proba_RFC_S = clf_scaled.predict_proba(X_test_scaled)[:,1]
fpr, tpr, thresholds = roc_curve(y_test_numeric, y_proba_RFC_S)
auc_RFC_S = auc(fpr, tpr)

# Diplay results
print(f"Confusion Matrix for Scaled RFC model:")
print(f"{confusion_matrix(y_true, y_pred_RFC_S)}")
print(f"true positives: {tp}")
print(f"true negatives: {tn}")
print(f"false negatives: {fn}")
print(f"false positives: {fp}")
print(f"Classification Report for Scaled RFC model:")
print(classification_report(y_true, y_pred_RFC_S))
print(f"Area Under the Curve for Scaled RFC model:")
print(auc_RFC_S)

Confusion Matrix for Scaled Log. Reg. model:
[[1752  599]
 [ 541 1810]]
true positives: 1810
true negatives: 1752
false negatives: 541
false positives: 599
Classification Report Scaled Log. Reg. model:
              precision    recall  f1-score   support

           0       0.76      0.75      0.75      2351
           1       0.75      0.77      0.76      2351

    accuracy                           0.76      4702
   macro avg       0.76      0.76      0.76      4702
weighted avg       0.76      0.76      0.76      4702

Area Under the Curve Scaled Log. Reg. model:
0.8350848829271814
 
------------------------------------------------------
 
Confusion Matrix for Scaled RFC model:
[[1960  391]
 [1264 1087]]
true positives: 1087
true negatives: 1960
false negatives: 1264
false positives: 391
Classification Report for Scaled RFC model:
              precision    recall  f1-score   support

           0       0.61      0.83      0.70      2351
           1       0.74      0.46      0.57 

#### Analyze scaled model performance and compare to prediction

As predicted, scaling the data had virtually no impact on the results of the RFC model, with the training scores only increasing from 0.647 to 0.648, and the scores for the confusion matrix, classification report, and area under the curve having similarly tiny changes. 

Scaling also improved the results of the logistic regression model as I thought it would, though much more than I expected as it now out-performs the RFC model in testing/accuracy scores (0.76, vs 0.65), f1-scores (avg of 0.76, vs 0.64), and AUC (score of 0.84 vs 0.70). Interestingly, the scores in the log. reg. model's classification report were also evened out, with the precision, recall, and f1-scores for both low- and high-risk cases all being around 0.76 now rather than having scores ranging from 0.35 to 0.77 and about 0.10 difference or more between low- and high-risk scores. 

Should an individual investor be looking to use on of these models to decide which loans to fund, I'd mostly likely recommend using the logistic regression model with scaled data rather than the random forest classifier model as it has better and more balanced performance scores overall. However, the RFC model may still be useful to those who value specifity (in this case, the model's ability to accurately classify high-risk loans) over sensitivity (its ability to accurately classify low-risk loans). Even after scaling, though the RFC model's sensitivity score (0.46) is lower than the logistic regression model's (0.77), it's specificity score is higher (0.83 vs 0.75), and therefore may be preferable for who invetors would rather take more conservative risk decisions. 